In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import csv
import concurrent.futures


In [2]:
organisms = pd.read_csv('orgs.txt', sep = '\t')
taxon_ids = organisms['taxon_oid']
taxon_ids

0        3300026111
1        3300064906
2        3300000615
3        3300003789
4        3300038838
            ...    
20944    3300042066
20945    3300002188
20946    3300011176
20947    3300002795
20948    3300033098
Name: taxon_oid, Length: 20949, dtype: int64

In [4]:
import concurrent.futures
from bs4 import BeautifulSoup
import requests

session = requests.Session()

def scrape_project_batch(taxon_ids):
    results = []
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = [executor.submit(scrape_project, id_) for id_ in taxon_ids]
        for future in concurrent.futures.as_completed(futures):
            try:
                result = future.result()
                results.append(result)
            except Exception as e:
                print(f"Error: {e}")
                # create list of error taxon_ids instead
    return results

def scrape_project(id_):
    url = 'https://img.jgi.doe.gov/cgi-bin/m/main.cgi?section=TaxonDetail&page=taxonDetail&taxon_oid={}'.format(id_)
    response = session.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    if header:
        headings = soup.find_all('th', class_='subhead')
        features = [heading.get_text() for heading in headings if heading.get_text() != 'Sample Information']

    row_data = {}
    rows = soup.find_all('tr', class_='img')
    for row in rows:
        column = row.find('th', class_='subhead')
        if column is not None:
            header_text = column.get_text()
            row_data[header_text] = row.find('td', class_='img').get_text()

    return row_data

header = True 
data = scrape_project_batch(taxon_ids)

In [5]:
df = pd.DataFrame(data)
pd.DataFrame.to_csv(df, 'metadata.csv', index=False)